# **NER 사전 만들기**
https://pypi.org/project/mdict-utils/
1. **MDX 사전파일을** 활용하여 객체에 구분하기

## **1 Sqlite3 데이터 불러오기**
sqlite3 데이터 불러오기

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('backup/kordict.db')
resp = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tableNames = [name[0]  for name in resp]
tableNames

['meta', 'mdx']

In [2]:
%%time
df = pd.read_sql_query("SELECT * FROM {}".format(tableNames[1]), conn)
itemIndex = df.entry.values.tolist()
itemData  = df.paraphrase.values.tolist()
print(len(itemIndex), itemIndex[:10], df.shape)
df.head()

516490 ['ᄛ', 'ᄻ', 'ᆐ', 'ㄱ', 'ㄱ', 'ㄱㄴㄷ순', 'ㄱㄴㄷ차례', 'ㄱㄴ순', 'ㄱㄴ차례', 'ㄱ자'] (516490, 2)
CPU times: user 1.19 s, sys: 259 ms, total: 1.45 s
Wall time: 1.52 s


,entry,paraphrase
0,ᄛ,<font color=crimson size=+2><b>ᄛ</b></font><br...
1,ᄻ,<font color=crimson size=+2><b>ᄻ</b></font><br...
2,ᆐ,<font color=crimson size=+2><b>ᆐ</b></font><br...
3,ㄱ,<font color=crimson size=+2><b>ㄱ</b></font><br...
4,ㄱ,<font color=crimson size=+2><b>ㄱ</b></font><br...


## **2 자료 전처리**
1. regex 를 활용하여 가치있는 **명사내용** 추출하기
1. "「1」" 내용은 동일한 개념의 단어를 정리한 것
1. sentence 별로 내용의 확인 및 구분이 필요

```html
<font color=crimson size=+2><b>대상행동</b></font>
<br><br>
<font COLOR="#00008B"><b>대상^행동(代償行動)</b></font>
『심리』
<br>
자기가 요구하는 바를 얻지 못할 때 그와 비슷한 다른 대상으로 만족을 채우려는 행동. 
어떤 대상에 의하여 일어나는 긴장을 다른 대상으로 해소하는 방어 기제의 하나이다. 
≒보상 작용
<br>
「1」ㆍ보상 행동.\r\n'
 ```

In [3]:
# 색인정보 : 51만개의 단어사전 데이터
itemIndex = df.entry.values.tolist()
itemData  = df.paraphrase.values.tolist()
len(itemIndex), itemIndex[:10]

(516490, ['ᄛ', 'ᄻ', 'ᆐ', 'ㄱ', 'ㄱ', 'ㄱㄴㄷ순', 'ㄱㄴㄷ차례', 'ㄱㄴ순', 'ㄱㄴ차례', 'ㄱ자'])

## **3 HTML 태그 제거 및 본문선별**
1. **/font** 태그가 제목 이외에, 세부내용에도 포함됩니다.
1. 때문에 이를 무조건 적용하기엔 문제가 있어서 Tag 구조를 명확하게 파악하기
1. 우선 구조에 대한 형태를 몇가지로 확인한 뒤 작업을 진행하기

```python
'<title.*?>(.+?)</title>'  # 특정태그
'<.*?>' # 모든 태그
```

In [5]:
search_query = "대리인"
num = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
num

105924

In [6]:
tempXml = itemData[num].lower()
from bs4 import BeautifulSoup
html_source = BeautifulSoup(tempXml, 'html.parser').prettify()
html_source # print(html_source)

'<font color="crimson" size="+2">\n <b>\n  대리인\n </b>\n</font>\n<br/>\n<br/>\n<font color="#00008b">\n <b>\n  대리-인(代理人) [대ː--]\n </b>\n</font>\n<br/>\n<font color="#549606">\n <b>\n  「명사」\n </b>\n</font>\n<br/>\n<br/>\n<br/>\n<font color="#cb4a00">\n <b>\n  「1」\n </b>\n</font>\n다른 사람을 대신하는 사람.\n<br/>\n¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫\n<br/>\n<font color="#cb4a00">\n <b>\n  「2」\n </b>\n</font>\n『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\n'

In [7]:
import re
regex_html = '<.*?>'                 # html 태그내용만 추출 (게으른 수량자)
# regex_html = '<.*>'                 # html 태그내용만 추출 (탐욕적 수량자)
regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
regex_font = '<font.*?>(.+?)</font>' # html 추출 : font 태그 내용만 추출

# html 태그의 추출
f_html = re.findall(regex_html, tempXml) # HTML 태그내용
", ".join(f_html)

'<font color=crimson size=+2>, <b>, </b>, </font>, <br>, <br>, <font color="#00008b">, <b>, </b>, </font>, <br>, <font color="#549606">, <b>, </b>, </font>, <br>, <br>, <br>, <font color="#cb4a00">, <b>, </b>, </font>, <br>, <br>, <font color="#cb4a00">, <b>, </b>, </font>'

In [9]:
# 특정 font 태그 내부의 text 를 추출
f_font = re.findall(regex_font, tempXml)
["<font>{}</font>".format(_) for _ in f_font]
[_ for _ in f_font]

['<b>대리인</b>',
 '<b>대리-인(代理人) [대ː--]</b>',
 '<b>「명사」</b>',
 '<b>「1」</b>',
 '<b>「2」</b>']

In [12]:
# f_font 의 태그를 제거한 나머지
f_txt = [re.sub(regex_html,"", _)  for _ in f_font] 
f_txt

['대리인', '대리-인(代理人) [대ː--]', '「명사」', '「1」', '「2」']

In [10]:
f_full = re.findall(regex_text, tempXml) 
", ".join(f_full)

'<font color=crimson size=+2>, <b>, 대리인, </b>, </font>, <br>, <br>, <font color="#00008b">, <b>, 대리-인(代理人) [대ː--], </b>, </font>, <br>, <font color="#549606">, <b>, 「명사」, </b>, </font>, <br>, <br>, <br>, <font color="#cb4a00">, <b>, 「1」, </b>, </font>, 다른 사람을 대신하는 사람., <br>, ¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫, <br>, <font color="#cb4a00">, <b>, 「2」, </b>, </font>, 『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n'

In [10]:
# 「1」', '「2」 개념설명 1, 2
# ¶  문장의 예시
[_  for _ in f_full  if _ not in f_html]

['대리인',
 '대리-인(代理人) [대ː--]',
 '「명사」',
 '「1」',
 '다른 사람을 대신하는 사람.',
 '¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫',
 '「2」',
 '『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n']

In [13]:
# font 태그 내용을 제거한 나머지
[_  for _ in f_full  if _ not in f_html  if _ not in f_txt]

['다른 사람을 대신하는 사람.',
 '¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫',
 '『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n']

In [14]:
%%time
result = []
for _ in itemData:
    result += re.findall(regex_html, _) # HTML 태그내용
result      = list(set(result))
result_tags = []         # 결과 중 사진포함 내용을 제거한 나머지
regex_img   = '<img.*?>' # html 추출 : font 태그 내용만 추출
for _ in result:
    _ = re.sub(regex_img, "", _)
    if len(_) > 2:
        result_tags.append(_.lower())
len(set(result_tags)), sorted(set(result_tags))

CPU times: user 3.64 s, sys: 196 ms, total: 3.84 s
Wall time: 3.84 s


(16,
 ['< >',
  '< n이면 반드시 ',
  '</b>',
  '</center>',
  '</font>',
  '</h1>',
  '<b>',
  '<br>',
  '<center>',
  '<font color="#00008b">',
  '<font color="#549606">',
  '<font color="#cb4a00">',
  '<font color="#cc00ff">',
  '<font color=crimson size=+2>',
  '<h1>',
  '<span style="definition">'])

## **정규표현식 연습하기**
Regex Tutorials

In [13]:
import re
re.findall('.+', "sales.xls, test.xls")

['sales.xls, test.xls']

In [14]:
re.findall('(<br>|<br/>)', "<br> 텍스트 <br/>")

['<br>', '<br/>']

In [15]:
import re
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")
print(p.sub("\g<phone> \g<name>", "park 010-1234-1234"))

010-1234-1234 park


In [16]:
text = "$23.24  의 가격은 $69.23 까지 상승하였습니다"
p = re.compile(r"\$[0-9.]+")
p.findall(text)

['$23.24', '$69.23']

In [17]:
p = re.compile(r"(?<=\$)[0-9.]+")
p.findall(text)

['23.24', '69.23']

## **4 분류 작업용 함수 만들기**
3개의 분류를 활용하여 List 만들기, 이후 대표적인 대상만 남기고 줄이기
1. 타이들
1. 대분류
1. 소분류 1, 2

In [51]:
search_query = "대리인"
num     = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
tempXml = itemData[num].lower()
tempXml

'<font color=crimson size=+2><b>대리인</b></font><br><br><font color="#00008b"><b>대리-인(代理人) [대ː--]</b></font><br><font color="#549606"><b>「명사」</b></font><br><br><br><font color="#cb4a00"><b>「1」</b></font>다른 사람을 대신하는 사람.<br>¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫<br><font color="#cb4a00"><b>「2」</b></font>『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n'

In [52]:
import re
regex_html = '<.*?>'                 # html 태그내용만 추출
regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
regex_font = '<font.*?>(.+?)</font>' # html 추출 : font 태그 내용만 추출
f_full = re.findall(regex_text, tempXml) 
f_html = re.findall(regex_html, tempXml) # HTML 태그내용
f_test = [_  for _ in f_full  if _ not in f_html]
f_test

['대리인',
 '대리-인(代理人) [대ː--]',
 '「명사」',
 '「1」',
 '다른 사람을 대신하는 사람.',
 '¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫',
 '「2」',
 '『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n']

In [58]:
# (?<=\$)
test  = "「명사」「1」「동-사」"
t_txt = re.compile(r"「[·가-힣]+」")
t_txt = re.compile(r"「([·가-힣]+)」")
t_num = re.compile(r"「\d+」")
t_box = re.compile(r"「.*?」")
# t_box = re.compile(r"(?<=\「).*?(?<=\」)$")
# t_num.findall(test)
# t_txt.findall(test)
t_box.findall(test)

['「명사」', '「1」', '「동-사」']

In [54]:
search_query = "대리인"
num     = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
tempXml = itemData[num].lower()
tempXml

'<font color=crimson size=+2><b>대리인</b></font><br><br><font color="#00008b"><b>대리-인(代理人) [대ː--]</b></font><br><font color="#549606"><b>「명사」</b></font><br><br><br><font color="#cb4a00"><b>「1」</b></font>다른 사람을 대신하는 사람.<br>¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫<br><font color="#cb4a00"><b>「2」</b></font>『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n'

In [50]:
result, result_len = [], []
for _ in itemData:
    temp = t_txt.findall(_)
    temp_num = len(temp)
    result_len.append(temp_num)
    if temp_num > 1:
        result += temp
result =list(set(result))
len(result)

22

## **4 분류 작업용 함수 만들기 2**
1. "「.*?」" 를 기준으로 객체들을 구분 분류하기
1. 추후 숫자와 개념어를 구분하여 분류 재정의 하기

In [55]:
# 텍스트 인덱스값 찾기
search_query = "가결"
search_query = "대리인"
search_query = "서버"
num     = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
tempXml = itemData[num].lower()
tempXml

'<font color=crimson size=+2><b>서버</b></font><br><br><font color="#00008b"><b>서버02(server)</b></font><br><font color="#549606"><b>「명사」</b></font><br>『컴퓨터』<br>주된 정보의 제공이나 작업을 수행하는 컴퓨터 시스템. 서버는 클라이언트 시스템이 요청한 작업이나 정보의 수행 결과를 돌려준다.\r\n'

In [57]:
import re
regex_html = '<.*?>'                 # html 태그내용만 추출
regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
f_txt      = [re.sub(regex_html,"", _)  for _ in f_font] 
f_full     = re.findall(regex_text, tempXml)
t_temp     = [_  for _ in f_full  if _ not in f_html]
t_temp

['서버',
 '서버02(server)',
 '「명사」',
 '『컴퓨터』',
 '주된 정보의 제공이나 작업을 수행하는 컴퓨터 시스템. 서버는 클라이언트 시스템이 요청한 작업이나 정보의 수행 결과를 돌려준다.\r\n']

In [ ]:
# 첫번째 의미만 찾기 최소 4글자 이상인 단어를 추출

In [ ]:
t_category = re.compile(r"「([·가-힣]+)」")
t_midsep   = re.compile(r"「([·가-힣]+)」")


In [41]:
itemIndex[654]

'가결'

In [38]:
list(set(result_len))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [39]:
result

['「대명사」',
 '「부사」',
 '「준」',
 '「반」',
 '「명사」',
 '「본」',
 '「동사」',
 '「관형사·명사」',
 '「형용사」',
 '「비」']

In [ ]:
t_txt

In [26]:
result

['「높」',
 '「5」',
 '「 」',
 '「」',
 '「19」',
 '「24」',
 '「2」',
 '「수사」',
 '「3」',
 '「의존명사·조사」',
 '「6」',
 '「비」',
 '「보조형용사」',
 '「의존명사」',
 '「13」',
 '「수사·관형사」',
 '「동사·형용사」',
 '「감탄사·명사」',
 '「준」',
 '「보조동사」',
 '「25」',
 '「1 」',
 '「21」',
 '「조사」',
 '「1」',
 '「감탄사」',
 '「반」',
 '「어미」',
 '「9」',
 '「18」',
 '「20」',
 '「관형사」',
 '「8」',
 '「대명사」',
 '「22」',
 '「15」',
 '「명사·부사」',
 '「접사」',
 '「26」',
 '「4」',
 '「북」',
 '「형용사」',
 '「부사」',
 '「16」',
 '「17」',
 '「12」',
 '「Ⅰ」',
 '「11」',
 '「2 」',
 '「7」',
 '「명사」',
 '「참고 어휘」',
 '「23」',
 '「본」',
 '「동사」',
 '「관형사·명사」',
 '「14」',
 '「10」']

In [92]:
tok = re.compile('「.*?」')
len(tok.findall(itemData[896]))

2

In [93]:
itemIndex[896]

'가귀노름'

In [87]:
result = [no for no, _ in enumerate(itemData)  if len(tok.findall(_)) > 1]
len(result)

105658

In [88]:
result[:100]

[21,
 23,
 26,
 29,
 31,
 42,
 43,
 51,
 57,
 59,
 64,
 86,
 99,
 109,
 110,
 111,
 132,
 134,
 135,
 140,
 142,
 144,
 148,
 159,
 206,
 226,
 227,
 230,
 250,
 255,
 256,
 263,
 394,
 406,
 419,
 503,
 508,
 510,
 511,
 513,
 514,
 519,
 520,
 530,
 531,
 539,
 541,
 545,
 566,
 579,
 580,
 584,
 589,
 594,
 596,
 597,
 625,
 627,
 645,
 654,
 656,
 660,
 661,
 662,
 674,
 676,
 682,
 683,
 686,
 689,
 690,
 707,
 717,
 734,
 735,
 745,
 747,
 748,
 751,
 757,
 780,
 790,
 821,
 822,
 823,
 828,
 829,
 833,
 836,
 837,
 841,
 850,
 851,
 863,
 867,
 872,
 887,
 889,
 892,
 896]

In [75]:
tok = re.compile('「.*?」')
tok.findall("".join(f_test))

['「명사」', '「1」', '「2」']

In [ ]:
tok = re.compile('「.*?」')
for _ in f_test:
    t_temp = tok.findall(_)[0].replace("「","").replace("」","")
    if t_temp:
    

In [69]:
str("명사")

'명사'

In [71]:
str('1')

'1'

In [68]:

tok = tok.findall(f_test[3])[0].replace("「","").replace("」","")
print(tok)
# 유효한 값이 추출된 경우
if tok:
    # 숫자가 추출시 : 개념 갯수만큼 나눈다
    try: 
        no = int(tok)
        print(re.compile('「.*?」').split(" ".join(f_test[3:])))
    except:
        print("Error", tok)

1
['', ' 다른 사람을 대신하는 사람. ¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다./너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫ ', ' 『법률』대리를 할 수 있는 지위에 있는 사람. 의사 능력이 있는 사람이어야 하며 대리인이 한 행위의 효과는 본인에게 귀속된다. 민법에는 법정 대리인과 임의 대리인이 있으며, 민사 소송에는 소송 대리인, 형사 소송에는 변호인이 있다. ≒대리자.\r\n']


In [55]:
temp = """「1」 다른 사람을 대신하는 사람.¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다.
/너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫「2」『법률』대리를 할 수"""

In [59]:
temp.split('「.*?」')

['「1」 다른 사람을 대신하는 사람.¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다.\n/너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫「2」『법률』대리를 할 수']

['',
 ' 다른 사람을 대신하는 사람.¶ 본인이 직접 가지 않고 대리인을 시켜 가게 했다.\n/너를 내 대리인으로 그 상회에 근무하도록 부탁했지.≪황석영, 무기의 그늘≫',
 '『법률』대리를 할 수']

In [ ]:
l = re.compile("(?<!^)\s+(?=[A-Z])(?!.\s)").split(s)

In [58]:
# Import the regex module
____

# Write a pattern to match sentence endings: sentence_endings
sentence_endings = r"[___]"

# Split my_string on sentence endings and print the result
print(re.____(____, ____))

# Find all capitalized words in my_string and print the result
capitalized_words = r"[___]\w+"
print(re.____(____, ____))

# Split my_string on spaces and print the result
spaces = r"___"
print(re.____(____, ____))

# Find all digits in my_string and print the result
digits = r"___"
print(re.____(____, ____))


NameError: name '____' is not defined

In [ ]:
for _ in f_test:
    # 분류기준의 추출
    if 

In [ ]:
result = []
for _ in itemData:
    temp = ''
    tok  = re.compile(r"(「.*」)")
    for t in tok.findall(_):
        if len(t) < 10:
            t = t.replace("「","")
            temp += ","+ t.replace("」","")
    result.append(temp)
len(result)

In [24]:
from konlpy.tag  import Hannanum

In [28]:
Hannanum().nouns("돼지김치찌개")

['돼지김치찌개']

In [29]:
Hannanum().nouns("소고기파스타")

['소고기파스타']

In [31]:
Hannanum().nouns("김치만두")

['김치만두']

In [33]:
Hannanum().pos("김치만두를 프라이팬에 굽습니다")

[('김치만두', 'N'),
 ('를', 'J'),
 ('프라이팬', 'N'),
 ('에', 'J'),
 ('굽', 'P'),
 ('습니다', 'E')]

In [44]:
result_html = " ".join(list(set(result)))

In [45]:
test = "<img src='hc513sy.jpg' width='300'> 테스트 추출"

In [43]:
list(set(result))

["<img src='hc513sy.jpg' width='300'>",
 "<img src='ja197py.jpg' width='300'>",
 "<img src='julgi01.jpg' width='300'>",
 "<img src='cb548py.jpg' width='300'>",
 "<img src='h00225.jpg' width='300'>",
 "<img src='jb520sy.jpg' width='300'>",
 "<img src='bb522sy.jpg' width='300'>",
 "<img src='hf566py.jpg' width='300'>",
 "<img src='ba523sy.jpg' width='300'>",
 "<img src='gf095py.jpg' width='300'>",
 "<img src='cc182sx.jpg' width='300'>",
 "<img src='ab088py.jpg' width='300'>",
 "<img src='gd139sx.jpg' width='300'>",
 "<img src='ic125py.jpg' width='300'>",
 "<img src='af103sy.jpg' width='300'>",
 "<img src='gb165sx.jpg' width='300'>",
 "<img src='fb587sz.jpg' width='300'>",
 "<img src='hc143sx.jpg' width='300'>",
 "<img src='la002sx.jpg' width='300'>",
 "<img src='ia585sy.jpg' width='300'>",
 "<img src='ha190py.jpg' width='300'>",
 '<img align=absmiddle src="chn_10016.gif">',
 "<img src='ab149sx.jpg' width='300'>",
 "<img src='gb540sy.jpg' width='300'>",
 "<img src='ic083sx.jpg' width='300

In [ ]:
f_full

## **3 HTML 태그 제거 및 본문선별**
1. **/font** 태그가 제목 이외에, 세부내용에도 포함됩니다.
1. 때문에 이를 무조건 적용하기엔 문제가 있어서 Tag 구조를 명확하게 파악하기
1. 우선 구조에 대한 형태를 몇가지로 확인한 뒤 작업을 진행하기

```python
'<title.*?>(.+?)</title>'  # 특정태그
'<.*?>' # 모든 태그
```

In [ ]:
tokenizer = re.compile("<*h4[^>]*>(.*?)<\s*/\s*h4>/g")

In [ ]:
tokenizer.findall('<h4 class="sds">And more ...</h4>')

In [ ]:
re.findall

In [ ]:
regex = re.compile("/<(.|\n)*?>/")

In [ ]:
regex.finditer("<p>test</p>")

In [ ]:
var body = ;
var result = body.replace(regex, "");

In [ ]:
idxNo = 12004
itemTemp = itemData[idxNo].lower()
toknizer = re.compile('<.*?>')
toknizer_class = re.compile('class="[a-zA-Z0-9:;\.\s\(\)\-\,]*"')
# re.sub(toknizer, '', itemTemp)
re.findall(toknizer, itemTemp)[6]

In [ ]:
re.findall(toknizer_class, re.findall(toknizer, itemTemp)[6])

In [ ]:
temp = '<font color="#00008b">'

In [ ]:
tok = re.compile('class="[a-zA-Z0-9:;\.\s\(\)\-\,]*"')

In [ ]:
tok.findall(temp)

In [ ]:
import re
re.findall(r"[ <\w>]+", itemTemp)

In [ ]:
idxNo

In [ ]:
# 큰 분류 3개 적용하기 : Item name, 중분류, 본문내용
idxNo = 106704
itemList = itemData[idxNo].split("</font>")
itemList

In [ ]:
# 큰 분류 3개 적용하기 : Item name, 중분류, 본문내용
idxNo = 106704
itemList = itemData[idxNo].split("</font>")
itemList

In [ ]:
import re
# html 태그내용 제거하는 함수
def remove_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
remove_html(itemList[1])

In [ ]:
itemList[-1].split('<br>')

## **4 본문의 분류내용 검색하기**
본문 텍스트는 Tag, 내용, 동의어 등 여러 정보가 포함되어 있다
1. 본문의 여러자료들을 개별 구성요소에 따라 구분하기
1. 구분한 뒤, Json 형식으로 저장해서 자료를 찾기 용이하도록 작업하기

```python
<font color=crimson size=+2>
    <b>가로물리다</b>     # 제목
</font>
<br><br>
<font color="#00008b">
    <b>가로물-리다02</b>  # 중간제목
</font>
<br>:〔-리어[-어/-여](-려),-리니〕
<br><br>
<font color="#549606">
    <b>「동사」</b>      # 별도 칼라로 구분도 함..
</font>                
<br><br>
【…에/에게 …을】
<br>
‘가로물다’의 사동사.\r\n
```

In [ ]:
itemDocs = [_.split("</font>")[2:]  for _ in itemData]
len(itemDocs)

In [ ]:
import re
itemResult = []
for item in itemDocs:
    temp = []
    item = item.split('<br>')[:4]
    for _ in item:
        if len(_) < 5:
            _temp = "".join(re.findall("\w+", _)).strip()
            if _temp:
                temp.append(_temp)
    itemResult.append(temp)

In [ ]:
itemData[2004].lower()

In [ ]:
itemData[2004].split('</font>')

In [ ]:
itemResult[2000:3000]

In [ ]:
temp = ''
import re
for _ in itemData:
    tok  = re.compile(r"(『.*』)")
    for t in tok.findall(_):
        if len(t) < 10:
            t = t.replace("『","")
            temp += ","+ t.replace("』","")

print(len(set(temp.split(","))))
tags = list(set(temp.split(",")))
",".join(tags)

In [ ]:
# 유효값 포함단어는 34만개 (이 방식이 제일 빠르다)
count = [1  for _ in result if _]
len(count)

In [ ]:
num_Idx = [no  for no, _ in enumerate(result) if _ == ",1"]
len(num_Idx)

In [ ]:
itemIndex[num_Idx[10]]

In [ ]:
num_Idx[10]

# 유효값 포함단어는 34만개
valid_token = [_ for _ in result  if len(_) > 1]
valid_token = list(set(valid_token))
valid_token[:5]

# **DataFrame 에서 바로 작업하기**
위 전처리 과정을 모두 정리한 뒤, 작업은 DataFrame 에서 바로 추출 요약을 하자!!
## **1 DataFrame 정리**

In [ ]:
print(len(set(temp.split(","))))
tags = list(set(temp.split(",")))
",".join(tags)

In [ ]:
# 음식 관련된 내용은 "명사" 내부에 포함

In [ ]:
temp = ''
tok  = re.compile(r"(『.*』)")
for t in tok.findall(itemData[200]):
    t = t.replace("『","")
    temp += ","+ t.replace("』","")
temp

In [ ]:
tok = re.compile(r"『.*』")
temp = tok.findall(itemData[20])
temp

In [ ]:
itemData[200000]

In [ ]:
%%time
import pickle
with open('data.pk', 'rb') as handle:
    mdkList = pickle.load(handle)

mdkList = [ list(map(lambda x:str(x), _)) for _ in mdkList]
mdkList = [" ".join(_)  for _ in mdkList] # 나눠진 객체를 1줄로 묶기

In [ ]:
%%time
import re
tags = []
# [list (tuple) (tuple)] 결과를 출력
tok = re.compile(r"(「.*」)|(『.*』)")  
for t in mdkList:
    txt = ""     
    # 1줄 처리시작
    for _ in tok.findall(t):
        temp = ",".join(_)
        txt += temp
    
    # 괄호기호 제거하기
    for _ in ["「","」","『","』"]:
        txt = txt.replace(_, "")
        
    # 불필요한 공백내용 제거하기
    txt = txt.replace(",", " ").strip()
    txt = [_  for _ in txt.split(" ")  if _]
    tags.append(",".join(txt))
len(tags)

In [ ]:
result[2].split(' ')

In [ ]:
result[0]

In [ ]:
result[1]

In [ ]:
result[2]

In [ ]:
tok.findall(mdkList[100])

In [ ]:
tok.findall(mdkList[300])

In [ ]:
txt.split(',')

## **2 사전파일 살펴보기**
**paraphrase** 내용을 **사용자가 필요한 형태로** 변경하기
```html
<font color=crimson size=+2>
    <b>ㄹ라</b>
</font>
<br>
<br>
<font color="#00008b">
    <b>-ㄹ라03</b>
</font>
<br>
<font color="#549606">
    <b>「어미」</b>
</font>
<br>『방언』<br>‘-려04’의 방언(경상).\r\n
```

In [ ]:
# 색인정보 : 51만개의 단어사전 데이터


In [ ]:
data = itemData[0]
data = data.lower()  # html Tag 전처리
print(data)

# 간단하게 처리한 뒤 유사내용 묶기
from lxml.html import fromstring
fromstring(data).xpath('//text()')

## **3 Regex를 활용하여 구분하기**
본문에 "\<br\>" 내용이 많아서 "//text()" 를 사용하면 컬럼갯수가 꼬여버린다
regex, lxml 을 사용하면 좋겠지만 경우의 수가 많아서 모두 처리하지 않으면 안됨..

In [ ]:
data = itemData[2945]
data = data.lower()  # html Tag 전처리
data

In [ ]:
import re
re.findall(r'<font color=crimson (.*?)</font>', data, re.DOTALL)

In [ ]:
remove_html_tags(data)

In [ ]:
from bs4 import BeautifulSoup
dom = BeautifulSoup(data, "lxml")
bs_dom = dom.find_all('font')
bs_txt = [str(_.get_text()) for _ in bs_dom]
bs_txt

In [ ]:
bs_result = list(map(lambda x : str(x),  bs_dom))
bs_result[0]

In [ ]:
bs_result = list(map(lambda x : str(x),  bs_dom))
bs_result[1]

In [ ]:
re.sub('color="#00008b">', "", data)

In [ ]:
re.sub("<font", "", data)

In [ ]:
"".join(dom.find_all('font'))

In [ ]:
import re # 강결합으로 정치큰 부분 제거하기
re.sub('<br><font color=.*</font>', "",data)

In [ ]:
_ = re.sub('<font .*</font><br>', "",data).split("<br>")
_[0], _[1].replace("\r\n", "")

In [ ]:
data

In [ ]:
re.findall(r"", )

## **3 함수의 적용**
작업용 함수를 작성한 뒤 응용하기

In [ ]:
from lxml.html import fromstring
def exportTxt(data):
    yield fromstring(data).xpath("//text()")

# result = list(map(lambda x: next(exportTxt(x)), itemData[:200000]))
# result = list(map(lambda x: next(exportTxt(x)), itemData[200000:400000]))
result = list(map(lambda x: next(exportTxt(x)), itemData[400000:]))

import pickle
# Store data (serialize)
with open('data3.pk', 'w') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

## **4 Pickle 내용 합치기**
작업용 함수를 작성한 뒤 응용하기

In [ ]:
from glob import glob
pklist = sorted(glob('data?.pk'))

# Load data (deserialize)
import pickle
result = []
for _ in pklist:
    with open(_, 'rb') as handle:
        result.extend(pickle.load(handle))

In [ ]:
with open('data.pk', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

## **5 사전자료 전처리 작업**
작업용 함수를 작성한 뒤 응용하기

In [ ]:
# tag 내용만 추출하기
tags = [_[-2]  for _ in mdkList]
len(set(tags))

In [ ]:
list(set(tags))

In [ ]:
import re
re.findall(r"[^ 가-힣]+", mdkDict[:3][0][0])

In [ ]:
result[:4]

In [ ]:
%whos

In [ ]:
# http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%A0%9C%EB%84%88%EB%A0%88%EC%9D%B4%ED%84%B0-generator/

In [ ]:
def square_numbers(nums):
    for i in nums:
        yield i * i

my_nums = square_numbers([1, 2, 3, 4, 5])  #1
next(my_nums)

In [ ]:
def foo(a, b):
    if a < 3:
        yield a, b
        a += 1
        for a, v in foo(a, b):
            yield a, v
    else:
        yield a, b

for a, v in foo(1, 2):
    print(a, v)

In [ ]:
itemData

In [ ]:
import re # 강결합으로 정치큰 부분 제거하기
re.sub('<br><font .*</font><br>', "",data)

In [ ]:
_ = re.sub('<font .*</font><br>', "",data).split("<br>")
_[0], _[1].replace("\r\n", "")

In [ ]:
import re
from lxml.html import fromstring
def exportMdx(data):
    data = data.lower()  # html Tag 전처리
    _ = re.sub('<font .*</font><br>', "",data).split("<br>")
    return fromstring(data).xpath('//font//text()'), (_[0], _[1].replace("\r\n", ""))

In [ ]:
result = []
from tqdm import tqdm
for _ in tqdm(itemData):
    result.append(exportMdx(_))


In [ ]:
def exportTag(data):
    temp = data.lower()
    temp = fromstring(temp).xpath('//font[@color="#549606"]//text()')
    return temp.strip()

from lxml.html import fromstring
from tqdm import tqdm
result = []
for _ in tqdm(itemData):
    result.append(exportTag(_))
tags = list(set(result))

In [ ]:
tags

In [ ]:
temp = str(df.entry[0])

In [ ]:
temp